In [561]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
# import missingno as msno
import copy

In [562]:
year_name ='U1398'
excel_file_path = f'{year_name}.xlsx'
excel_sheets_dict_2 = pd.read_excel(excel_file_path, sheet_name=None)

In [563]:
excel_sheets_dict = copy.deepcopy(excel_sheets_dict_2)

In [564]:
dfs = list(excel_sheets_dict.keys())
dfs

['U1398Data',
 'U1398P1',
 'U1398P2',
 'U1398P3S01',
 'U1398P3S02',
 'U1398P3S03',
 'U1398P3S04',
 'U1398P3S05',
 'U1398P3S06',
 'U1398P3S07',
 'U1398P3S08',
 'U1398P3S09',
 'U1398P3S10',
 'U1398P3S11',
 'U1398P3S12',
 'U1398P3S13',
 'U1398P3S14',
 'U1398P4S01',
 'U1398P4S02',
 'U1398P4S03',
 'U1398P4S04']

In [565]:
# cols_with_Missing = []
# for sheet in dfs:
#     if excel_sheets_dict[sheet].isna().sum().any():
#         cols_with_Missing.append(sheet)
#
# cols_with_Missing

In [566]:
R1401P1 = excel_sheets_dict[f'{year_name}P1']
R1401P1.head()

,Address,member,relation,gender,age,literacy,studying,degree,occupationalst,maritalst
0,10004004223,1,Head,Male,66,literate,No,Diploma,IncomeWOJob,Married
1,10004004223,2,Spouse,Female,61,literate,No,College,IncomeWOJob,Married
2,10004004220,1,Head,Male,36,literate,No,Elemantry,employed,Married
3,10004004220,2,Spouse,Female,33,literate,No,Secondary,Housewife,Married
4,10004004220,3,Child,Female,12,literate,Yes,Elemantry,Student,Single


In [567]:
pd.DataFrame({'Sum_of_MN':R1401P1.isna().sum(),'Mean_of_MN' : (R1401P1.isna().mean()* 100)})

,Sum_of_MN,Mean_of_MN
Address,0,0.000000
member,0,0.000000
relation,0,0.000000
gender,0,0.000000
age,0,0.000000
literacy,6099,8.936918
studying,13344,19.553081
degree,13345,19.554546
occupationalst,10472,15.344714
maritalst,10472,15.344714


In [568]:
R1401P1.literacy.unique()

array(['literate', nan, 'illiterate'], dtype=object)

In [569]:
R1401P1[(R1401P1.literacy.isna())].age.value_counts()

age
5    1177
4    1152
3    1114
2    1060
1     863
0     706
6      27
Name: count, dtype: int64

we dont want any condition on age for literacy because they are all kids and we can impute with illitrate


In [570]:
R1401P1.loc[(R1401P1.literacy.isna()),'literacy']='illiterate'

In [571]:
R1401P1.isna().sum() # No missing value

Address               0
member                0
relation              0
gender                0
age                   0
literacy              0
studying          13344
degree            13345
occupationalst    10472
maritalst         10472
dtype: int64

`literacy` column done

`studying` column

In [572]:
pd.DataFrame({'Sum_of_MN':R1401P1.isna().sum(),'Mean_of_MN' : (R1401P1.isna().mean()* 100)})

,Sum_of_MN,Mean_of_MN
Address,0,0.000000
member,0,0.000000
relation,0,0.000000
gender,0,0.000000
age,0,0.000000
literacy,0,0.000000
studying,13344,19.553081
degree,13345,19.554546
occupationalst,10472,15.344714
maritalst,10472,15.344714


In [573]:
R1401P1.studying.unique()

array(['No', 'Yes', nan], dtype=object)

In [574]:
R1401P1.studying.value_counts(dropna=False)

studying
No     39885
Yes    15016
NaN    13344
Name: count, dtype: int64

In [575]:
R1401P1[R1401P1.studying.isna()].head()

,Address,member,relation,gender,age,literacy,studying,degree,occupationalst,maritalst
5,10004004220,4,Child,Male,4,illiterate,NaN,NaN,NaN,NaN
15,10009008114,3,Child,Male,5,illiterate,NaN,NaN,NaN,NaN
16,10009008114,4,Child,Female,5,illiterate,NaN,NaN,NaN,NaN
26,10002002810,1,Head,Female,88,illiterate,NaN,NaN,IncomeWOJob,Widowed
39,10001000214,4,Child,Female,4,illiterate,NaN,NaN,NaN,NaN


In [576]:
R1401P1.loc[R1401P1.degree.isna()&(R1401P1.studying.isna()),['studying','degree']].isna().sum()

studying    13344
degree      13344
dtype: int64


میشه نتیجه گرفت دو ستون میزان تحصیلات و درس خواندن را با سواد نداشتن کامل پر کرد چه بچه باشه و چه نباشه

In [577]:
R1401P1.loc[R1401P1.studying.isna() & (R1401P1.degree.isna()) & (R1401P1.literacy=='illiterate'),'studying']='No'

In [578]:
R1401P1.loc[R1401P1.studying.isna() & (R1401P1.literacy=='illiterate'),'studying']='No'

In [579]:
R1401P1.studying.value_counts(dropna=False)

studying
No     53229
Yes    15016
Name: count, dtype: int64

In [580]:
R1401P1.loc[R1401P1.studying.isna()]

,Address,member,relation,gender,age,literacy,studying,degree,occupationalst,maritalst


In [581]:
R1401P1.loc[(R1401P1.studying=='No') & (R1401P1.degree.isna()) & (R1401P1.literacy=='illiterate'),'degree']='NoneEducation'
R1401P1.isna().sum()

Address               0
member                0
relation              0
gender                0
age                   0
literacy              0
studying              0
degree                1
occupationalst    10472
maritalst         10472
dtype: int64

`degree` column Done!

### `occupationalst` 9083

In [582]:
R1401P1['occupationalst'].unique()


array(['IncomeWOJob', 'employed', 'Housewife', 'Student', nan,
       'unemployed', 'Other'], dtype=object)

In [583]:
R1401P1[R1401P1['occupationalst']== 'unemployed'].head(10)

,Address,member,relation,gender,age,literacy,studying,degree,occupationalst,maritalst
9,10009008108,4,Child,Female,27,literate,Yes,Master,unemployed,Single
10,10009008117,1,Head,Male,52,literate,No,Secondary,unemployed,Married
12,10009008117,3,Child,Male,21,literate,No,College,unemployed,Single
48,10013010623,1,Head,Male,65,literate,No,Elemantry,unemployed,Married
63,10013010623,3,Child,Male,25,illiterate,No,NoneEducation,unemployed,Married
97,10001001514,3,Child,Male,24,literate,No,Diploma,unemployed,Single
102,10013010610,1,Head,Male,35,literate,No,Bachelor,unemployed,Married
103,10013010610,2,Spouse,Female,35,literate,No,Bachelor,unemployed,Married
104,10013010608,1,Head,Male,29,literate,No,Diploma,unemployed,Married
116,10006005518,3,Child,Male,20,literate,No,Diploma,unemployed,Single


In [584]:
R1401P1.groupby('occupationalst').agg({'age':'mean'})

,age
occupationalst,
Housewife,42.730315
IncomeWOJob,62.685885
Other,37.611842
Student,15.231296
employed,40.406051
unemployed,29.305107


یانگین سن مشاغل مختلف م

In [585]:
R1401P1[R1401P1['occupationalst'].isna()].age.value_counts()

age
5    1177
4    1152
7    1143
8    1141
3    1114
9    1100
2    1060
6    1016
1     863
0     706
Name: count, dtype: int64

یبینیم باز اونهایی که بدون شغل هستند کودک بین 0 تا 9 سال هستندم

In [586]:
R1401P1[R1401P1['occupationalst'].isna() & (R1401P1.age <= 9)]['gender'].value_counts()

gender
Male      5414
Female    5058
Name: count, dtype: int64

جالب اینجاست همه اونهایی که حرفه اشون مشخص نیست کودک هستن حالا ازبین اینها بعضی هاشون باید دانش آموز باشند اما نیستن فقط اونهایی که در حال تحصیل هستن رو قرار میدیم دانش آموز


In [587]:
R1401P1.loc[(R1401P1.studying=='Yes') & (R1401P1.occupationalst.isna()),'occupationalst']='Student'

In [588]:
R1401P1.loc[(R1401P1.occupationalst.isna()),'age'].value_counts()

age
5    1177
4    1152
3    1114
2    1059
1     863
0     706
6     277
7      28
8      16
9      16
Name: count, dtype: int64

In [589]:
R1401P1.loc[(R1401P1.age.between(7,9)) & (R1401P1.studying=='No'),].Address.count()

60

کودکانی که  درس نخواندن و یا ترک تحصیل کرده اند(کودک کار)

In [590]:
R1401P1.loc[(R1401P1.occupationalst.isna()),'occupationalst']='under_work_age'

In [591]:
occup = R1401P1.loc[(R1401P1.literacy == 'illiterate') & (R1401P1.studying == 'No') & (R1401P1.degree=='NoneEducation'), :]
occup = occup[occup.occupationalst.isna()]
occup

,Address,member,relation,gender,age,literacy,studying,degree,occupationalst,maritalst


In [592]:
R1401P1['occupationalst'].value_counts()

occupationalst
employed          17882
Housewife         17183
Student           14423
IncomeWOJob        7354
under_work_age     6408
unemployed         3779
Other              1216
Name: count, dtype: int64

اینجا ما کلی همه کودکان چه اونهایی که سواد دارند و چه سواد ندارند را "زیرسن کار""در نظر گرفتیم


### `maritalst`  9082

In [593]:
R1401P1.maritalst.unique()

array(['Married', 'Single', nan, 'Widowed', 'Divorced'], dtype=object)

به نظر میاد اینجا بیوه و طلاق گرفته نباید یکی و سینگل درنظر گرفته بشن

In [594]:
R1401P1.loc[R1401P1.maritalst.isna(),'age'].value_counts()

age
5    1177
4    1152
7    1143
8    1141
3    1114
9    1100
2    1060
6    1016
1     863
0     706
Name: count, dtype: int64

با توجه به این نتایج مشخص میشه همه اونهایی که وضعیت ازدواجشون مشخص نیست کودک هستن و میشه یک لیبل مثل  under_age  بهشون نسبت داد

In [595]:
R1401P1.loc[R1401P1.age<10,'maritalst'].value_counts()

Series([], Name: count, dtype: int64)

ینجا یک نفر هست که سنش کمتر از 10 ساله و سینگل هم هست میخوایم ببینیم چه مشخصاتی داره مثلا دختره که سینگل شده یا خیر؟؟ا

In [596]:
R1401P1.loc[(R1401P1.age<10) & (R1401P1.maritalst=='Single'),].value_counts()

Series([], Name: count, dtype: int64)

ا توجه به اینکه 2 سالش هست میتونیم از سینگل برای زیر 10 سالها استفاده کنیمب

In [597]:
R1401P1.loc[R1401P1.maritalst.isna(),'maritalst']='Single'

In [598]:
pd.DataFrame({'Sum_of_MN':R1401P1.isna().sum(),'Mean_of_MN' : (R1401P1.isna().mean()* 100)})

,Sum_of_MN,Mean_of_MN
Address,0,0.000000
member,0,0.000000
relation,0,0.000000
gender,0,0.000000
age,0,0.000000
literacy,0,0.000000
studying,0,0.000000
degree,1,0.001465
occupationalst,0,0.000000
maritalst,0,0.000000


extract Male and Female count columns

In [599]:
R1401P1['Male_count'] = R1401P1['gender'].apply(lambda x: 1 if x == 'Male' else 0)
R1401P1['Female_count'] = R1401P1['gender'].apply(lambda x: 1 if x == 'Female' else 0)
R1401P1.drop(['gender'],axis=1,inplace=True)

In [600]:
R1401P1.head()

,Address,member,relation,age,literacy,studying,degree,occupationalst,maritalst,Male_count,Female_count
0,10004004223,1,Head,66,literate,No,Diploma,IncomeWOJob,Married,1,0
1,10004004223,2,Spouse,61,literate,No,College,IncomeWOJob,Married,0,1
2,10004004220,1,Head,36,literate,No,Elemantry,employed,Married,1,0
3,10004004220,2,Spouse,33,literate,No,Secondary,Housewife,Married,0,1
4,10004004220,3,Child,12,literate,Yes,Elemantry,Student,Single,0,1


In [601]:
cat_columns = ['relation','literacy','studying','degree','occupationalst',
               'maritalst']
le = LabelEncoder()
for column in cat_columns:
    R1401P1[column] = le.fit_transform(R1401P1[column].astype(str))

R1401P1


,Address,member,relation,age,literacy,studying,degree,occupationalst,maritalst,Male_count,Female_count
0,10004004223,1,2,66,1,0,2,1,1,1,0
1,10004004223,2,8,61,1,0,1,1,1,0,1
2,10004004220,1,2,36,1,0,3,4,1,1,0
3,10004004220,2,8,33,1,0,9,0,1,0,1
4,10004004220,3,0,12,1,1,3,3,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...
68240,13001374515,5,0,15,1,1,9,3,2,1,0
68241,13006382723,4,0,16,1,1,2,3,2,1,0
68242,13006382723,5,0,8,1,1,3,3,2,0,1
68243,13001377517,1,2,61,1,0,3,1,1,1,0


In [602]:
new_df =R1401P1.groupby('Address').agg({'member':'count','Male_count':'sum','Female_count':'sum','age':'mean','literacy':'mean','studying':'mean',
                                        'degree':(lambda  x:x.value_counts().index[0]),
                                        'occupationalst':(lambda  x:x.value_counts().index[0]),'maritalst':(lambda  x:x.value_counts().index[0])}).reset_index()
new_df

,Address,member,Male_count,Female_count,age,literacy,studying,degree,occupationalst,maritalst
0,10001000108,5,2,3,46.60,1.000000,0.000000,5,0,2
1,10001000111,3,2,1,27.00,1.000000,0.333333,2,4,1
2,10001000114,1,0,1,49.00,1.000000,0.000000,2,1,3
3,10001000117,4,2,2,38.50,1.000000,0.000000,2,0,1
4,10001000120,4,1,3,49.75,1.000000,0.250000,3,0,1
...,...,...,...,...,...,...,...,...,...,...
19893,13006383810,3,2,1,53.00,0.666667,0.000000,3,4,1
19894,13006383814,2,1,1,25.00,1.000000,0.500000,2,4,0
19895,13006383815,1,1,0,52.00,1.000000,0.000000,4,4,2
19896,13006383820,5,2,3,24.20,0.800000,0.400000,3,3,2


In [603]:
new_df.to_csv(f'{year_name}P1.csv',index=False)

In [604]:
R1401P1 = pd.read_csv(f'{year_name}P1.csv')
R1401P1

,Address,member,Male_count,Female_count,age,literacy,studying,degree,occupationalst,maritalst
0,10001000108,5,2,3,46.60,1.000000,0.000000,5,0,2
1,10001000111,3,2,1,27.00,1.000000,0.333333,2,4,1
2,10001000114,1,0,1,49.00,1.000000,0.000000,2,1,3
3,10001000117,4,2,2,38.50,1.000000,0.000000,2,0,1
4,10001000120,4,1,3,49.75,1.000000,0.250000,3,0,1
...,...,...,...,...,...,...,...,...,...,...
19893,13006383810,3,2,1,53.00,0.666667,0.000000,3,4,1
19894,13006383814,2,1,1,25.00,1.000000,0.500000,2,4,0
19895,13006383815,1,1,0,52.00,1.000000,0.000000,4,4,2
19896,13006383820,5,2,3,24.20,0.800000,0.400000,3,3,2
